In [1]:
import pandas as pd
import numpy as np
import re
from zoneinfo import ZoneInfo  # Python 3.9+
from pathlib import Path

# -----------------------------
# تنظیمات ورودی/خروجی
# -----------------------------
INPUT_FILE = "power_code.xlsx"           # نام فایل ورودی
INPUT_SHEET = 0                          # نام یا ایندکس شیت (0 یعنی اولین شیت)
OUTPUT_FILE = "power_code_clean.xlsx"    # نام فایل خروجی
TIMEZONE = "Asia/Tehran"                 # تایم‌زون برای محاسبه تایم‌استمپ

# -----------------------------
# توابع کمکی
# -----------------------------

def parse_hour_to_timedelta(x):
    """
    مقدار ستون HOUR را به مدت‌زمان روز تبدیل می‌کند.
    ورودی‌ها قابل‌قبول:
      - عددی: 0..23 (مثلاً 15 => 15:00:00)
      - رشته‌های زمان: '15', '15:46', '15:46:19'
      - مقادیر خالی/نا‌معتبر => NaT
    خروجی: pd.Timedelta یا NaT
    """
    if pd.isna(x):
        return pd.NaT

    # اگر عددی بود
    if isinstance(x, (int, float, np.integer, np.floating)):
        # اگر float بود و دقیقه/ثانیه ندارد، فقط ساعت را می‌گیریم
        try:
            h = int(x)
            if 0 <= h <= 23:
                return pd.to_timedelta(f"{h}:00:00")
            else:
                # اگر خارج از بازه ساعت بود، تلاش می‌کنیم آن را به HH:MM:SS تعبیر کنیم
                return pd.to_timedelta(str(x), errors='coerce')
        except Exception:
            return pd.to_timedelta(str(x), errors='coerce')

    # اگر رشته بود
    s = str(x).strip()
    if s == "":
        return pd.NaT

    # حالت فقط ساعت (مثلاً '5' یا '15')
    if re.fullmatch(r"\d{1,2}", s):
        h = int(s)
        if 0 <= h <= 23:
            return pd.to_timedelta(f"{h}:00:00")
        else:
            return pd.to_timedelta(s, errors='coerce')

    # حالت 'HH:MM' یا 'HH:MM:SS'
    if re.fullmatch(r"\d{1,2}:\d{1,2}(:\d{1,2})?", s):
        return pd.to_timedelta(s, errors='coerce')

    # تلاش نهایی: هرچه هست به Timedelta تبدیل کن
    return pd.to_timedelta(s, errors='coerce')


def normalize_shamsi_date(x):
    """
    تاریخ شمسی را به صورت YYYY/MM/DD با دو رقمی‌کردن ماه/روز برمی‌گرداند.
    ورودی‌های قابل‌قبول: '1403/1/1', '1403-1-1', '1403/10/01', ...
    اگر ورودی نامعتبر باشد، همان را برمی‌گرداند.
    """
    if pd.isna(x):
        return x
    s = str(x).strip()
    # با جداکننده‌های / یا - تقسیم می‌کنیم
    parts = re.split(r"[/-]", s)
    if len(parts) != 3:
        return s
    year = parts[0].zfill(4)
    month = parts[1].zfill(2)
    day = parts[2].zfill(2)
    return f"{year}/{month}/{day}"


def combine_date_and_hour(date_series, hour_series):
    """
    ترکیب DATE_MILADI و HOUR:
    - DATE_MILADI به صورت تاریخ میلادی (مثلاً '5/22/2014')
    - HOUR به قالب‌های مختلف (عدد یا رشته زمان)
    خروجی: سری از نوع datetime64[ns] (نا-تاریخ‌ها => NaT)
    """
    # تبدیل تاریخ میلادی به datetime با فرض فرمت ماه/روز/سال (dayfirst=False)
    dates = pd.to_datetime(date_series, errors="coerce", dayfirst=False, infer_datetime_format=True)

    # تبدیل ساعت‌ها به Timedelta
    hours_td = hour_series.apply(parse_hour_to_timedelta)

    # جمع تاریخ و مدت‌زمان ساعت
    # اگر هرکدام NaT باشد، نتیجه NaT می‌شود
    combined = dates + hours_td

    return combined


def to_unix_timestamp(dt_series, tz_name=TIMEZONE):
    """
    dt_series: سری datetime بدون تایم‌زون (naive)
    1) به تایم‌زون ورودی لوکالایز می‌کنیم
    2) به UTC تبدیل می‌کنیم
    3) یونیکس تایم‌استمپ (ثانیه) را برمی‌گردانیم
    """
    tz = ZoneInfo(tz_name)
    # لوکالایز به تایم‌زون تعیین‌شده
    dt_local = dt_series.dt.tz_localize(tz, ambiguous="NaT", nonexistent="shift_forward")
    # تبدیل به UTC
    dt_utc = dt_local.dt.tz_convert(ZoneInfo("UTC"))
    # نانوثانیه به ثانیه
    return (dt_utc.view("int64") // 10**9).astype("Int64")  # استفاده از نوع nullable برای NaT


# -----------------------------
# اجرای پردازش
# -----------------------------

def main():
    # خواندن فایل
    if not Path(INPUT_FILE).exists():
        raise FileNotFoundError(f"فایل '{INPUT_FILE}' پیدا نشد.")

    df = pd.read_excel(INPUT_FILE, sheet_name=INPUT_SHEET)

    # اطمینان از وجود ستون‌های مورد نیاز
    required_cols = ["DATE_MILADI", "HOUR", "DATE_SHAMSI"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"ستون‌های زیر یافت نشد: {missing}")

    # 1) ساخت datetime استاندارد در DATE_MILADI با افزودن ساعت از HOUR
    combined_dt = combine_date_and_hour(df["DATE_MILADI"], df["HOUR"])

    # جایگزینی ستون DATE_MILADI با مقدار datetime جدید
    df["DATE_MILADI"] = combined_dt

    # 2) افزودن ستون TIMESTAMPS (یونیکس تایم‌استمپ)
    df["TIMESTAMPS"] = to_unix_timestamp(df["DATE_MILADI"], tz_name=TIMEZONE)

    # 3) نرمال‌سازی تاریخ شمسی به شکل YYYY/MM/DD با دو رقمی‌کردن ماه/روز
    df["DATE_SHAMSI"] = df["DATE_SHAMSI"].apply(normalize_shamsi_date)

    # ذخیره خروجی
    # اگر فایل خروجی موجود بود، روی آن می‌نویسیم
    df.to_excel(OUTPUT_FILE, index=False)
    print(f"پردازش کامل شد. فایل خروجی: {OUTPUT_FILE}")

if __name__ == "__main__":
    main()


C:\Users\pishva_r\AppData\Local\Temp\ipykernel_8776\1339015683.py:92: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dates = pd.to_datetime(date_series, errors="coerce", dayfirst=False, infer_datetime_format=True)
C:\Users\pishva_r\AppData\Local\Temp\ipykernel_8776\1339015683.py:117: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  return (dt_utc.view("int64") // 10**9).astype("Int64")  # استفاده از نوع nullable برای NaT


پردازش کامل شد. فایل خروجی: power_code_clean.xlsx
